# 深層学習- 1
参考：  
* [Udemy講座：「①米国AI開発者がやさしく教える深層学習超入門第一弾【Pythonで実践】」](https://www.udemy.com/course/deeplearning1/learn/lecture/40143418)
* https://axa.biopapyrus.jp/deep-learning/
* [パーセプトロン](https://axa.biopapyrus.jp/deep-learning/perceptron.html)
* [ニューラルネットワークの基礎](https://tutorials.chainer.org/ja/13_Basics_of_Neural_Networks.html)

## 深層学習とは
深層学習は機械学習のアルゴリズムの一種である。  
従来の機械学習手法（ロジスティック回帰やGBDTなど）は人が特徴量を作ってモデルに渡して学習させていたため、  
良い特徴量をいかに作るかが重要であった。  
一方、深層学習では入力データをモデルに渡すとモデル自身が特徴量を作って学習するため、  
良い特徴量を作ってくれるモデルをいかに作るかが重要という点で従来の機械学習手法と異なる。  
深層学習ではニューラルネットワーク(NN)という人間の神経を模したモデルを用いてデータを学習する。  
このNNの層を深くしていくことで精度が高いモデル構築することが出来る。  

深層学習の適用例は下記。  
昨今の生成AIも深層学習の応用であり、社会に非常に大きな影響を与えている技術の一つ。
* 音声認識
* 医用画像診断
* 顔認証
* 株価予想
* 画像生成

## 深層学習の文脈でよく使われる記号・表記
同様の意味でも統計学や深層学習以外の機械学習モデルで使用していた記号と深層学習で用いる記号は異なる。  
(ドキュメントや論文によって表記は異なる。)

### 例）線形回帰のモデル式
$\hat{y}=\hat{f}(x)=\theta_0 + \theta_1x$　→ $z = w_1x + b$  

機械学習における線形回帰の式のような線形変換（正しくはアフィン変換）の出力は、  
深層学習においては$z$などで表わす。  
機械学習では特徴量$x$に対する重みに相当する量は$\theta$で表すことが多かったが、  
深層学習ではそのまま$weight$の$w$を用いて表す。またbias項は$b$で表わす。  

### 例）シグモイド関数
$z=\frac{1}{1+e^{-x}}$  →  $\sigma(x)=\frac{1}{1+e^{-x}}$  

詳細は後述するが、この$\sigma(x)$はニューラルネットワーク（NN）における”活性化関数”で良く用いられる表記である。  
シグモイド関数はNNの活性化関数としてよく使われる。

### 例）ロジスティック回帰の式
$p(x)=\frac{1}{1+e^{-(\theta_0 + \theta_1)}}$  →  $p(x)=a=\sigma(z)=\sigma(w_1x + b)=\frac{1}{1+e^{-(w_1x + b)}}$  

活性化関数からの出力は$a$で表わすことが多い。  

### 例）softmax関数の式
$p_k(\hat{\bm{y}})=\frac{e^{\hat{y}_k}}{\sum_{k=1}^K e^{\hat{y}_k}}$  →  $\sigma(\bm{z})_j=\frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}}$

* $\hat{y}_k, e^{z_j}, e^{z_k}$にはkまたはj番目のクラス（赤など）を真の値として学習した線形回帰モデルの出力が入る。  
$\hat{\bm{y}}, \bm{z}$はクラスごとに値を持つベクトルであることに注意。  
例）$\bm{z}=(z_1, z_2, z_3)$


## Pytorch
深層学習ライブラリとしてはよくPytorchが用いられる。  
Tensorflowも一時期使われていたが、最近はPytorchが主流。  
深層学習に使用するデータはPytorchのTensorモジュールで取り扱う。

### - 計算グラフ
Pytorchではtensor同士の計算を行った際に”計算グラフ”(Computational Graph)というものを構築し、保持しておく機能がある。  
計算グラフは演算の計算過程のようなもので、計算グラフを用いることで任意の関数の勾配を容易に求めることが出来る。  
また、可視化することで人間にも視覚的に理解しやすい。詳細は講義資料参照。  
深層学習において、損失関数は重み$w_i$や重みによって決まる値を変数とする多変数関数だとみなし、  
これらの連鎖律（チェインルール）によって損失関数の勾配($\partial L/ \partial w_i$)を求める際に計算グラフを用いている。

参考：
https://manabitimes.jp/math/1303

## ミニバッチ勾配降下法（ミニバッチ学習）
深層学習ではメモリに乗り切らないような大規模データを学習させることが多い。  
そのため、基本的にすべての学習データを一気に学習させるのではなく、  
いくつかのデータを一つの塊（ミニバッチ）にして学習させていく。  
これを __ミニバッチ勾配降下法__ という。  

ミニバッチのサイズ（いくつのデータを塊とするか）は自由に選択できるが、GPUのメモリに乗る最大の$2^n$のバッチサイズを選ぶことが多い。  
ミニバッチサイズが1の場合は __確率的勾配降下法(SGD: Stochastic Gradient Descent)__ といい、全データの場合は __バッチ勾配降下法__ という。  
これらはミニバッチ勾配降下法の特別な場合といえる。（ミニバッチ勾配降下法をSGDと呼ぶこともあるので注意。）  
だが、下記のような難点があるので基本的にはミニバッチ勾配降下法で適切なデータサイズを検討することが主流。  

■ SGDとバッチ勾配降下法の難点：  
* SGD
    * 全データ数分の学習ループを回す必要があるので、学習に時間がかかる  
    ※一方、1データ分だけを変数に読み込むのでメモリ効率は良い
    * 損失が外れ値となるようなデータがある場合、パラメタが大きく更新されてしまい、収束から離れてしまうことがある  
    （最適化の際にノイズが入りやすい）  
    

* バッチ勾配降下法
    * 学習データが大きすぎるとメモリに乗り切らない
    * tensorなどの計算コストが高くなる
    * 全データから損失を算出するため、各データの損失を捉えきれず、収束しづらい(損失が小さくならない)  


※メモ  
* DL_notebook-3_Multinomial_Logistic_Regression.ipynbで実施したのはSGDに相当する
* ML_notebook-1(LinearRegression).ipynbの最急降下法は全データを使ったが、epoch数を大きくすることで収束させた